# Step 1: configuration

Input your walletPrivateKey here:

In [1]:
# make sure you configure mindlakesdk, please check env.py in the same folder
# please familar these configure, and you can read https://github.com/mind-network/mind-lake-sdk-python 
import env

print("check env.walletPrivateKey:", env.walletPrivateKey[0:5])
print("check env.mindLakeAppKey:", env.mindLakeAppKey[0:5])
print("check env.MINDLAKE_GATEWAY:", env.MINDLAKE_GATEWAY)

check env.walletPrivateKey: 2a776
check env.mindLakeAppKey: Kyunq
check env.MINDLAKE_GATEWAY: https://sdk.mindnetwork.xyz/node


Install depedancy and source code

In [2]:
# please "pip install mindlakesdk" if not installed
import mindlakesdk

# please "pip install minddatapack" if not installed, or git clone the source code
from minddatapack import DataPack

# please "pip install pandas" if not installed, it is used to display structure content in the notebook
import pandas

In [3]:
# check if example_data.csv file exists in the same folder, this demo will start to load this csv file.

# check if arweave_wallet.json file exists in the same folder, this demo needs to use arweave wallet to sign to send tx.

## Step 2. Local CSV -> MindLake
- Load data from a local CSV file without the metadata file, by defining each column manually.
- You can also load from a local backup into MindLake once you are familar, then no need to define each column. 

In [4]:
fn_local_csv = './example_data.csv'

# preview local csv
df = pandas.read_csv(fn_local_csv)
df

,uid,wallet_address,register_date
0,123,0x79Be957bf7e3003aFd0e78f04Bacbc93D3ef2fB7,2023-07-15 02:25:32.392441
1,124,0x79Be957bf7e3003aFd0e78f04Bacbc93D3ef2fB7,2023-07-15 02:25:32.392441
2,125,0x79Be957bf7e3003aFd0e78f04Bacbc93D3ef2fB7,2023-07-15 02:25:32.392441


In [5]:
dataPack1 = DataPack(env.walletPrivateKey)
result = dataPack1.loadFromCSVFileByDefineColumn(fn_local_csv, 
    [
        DataPack.Column('uid', DataPack.DataType.int4, False),
        DataPack.Column('wallet_address', DataPack.DataType.text, True),
        DataPack.Column('register_date', DataPack.DataType.timestamp, True)
    ])
assert result, result.message
df = pandas.DataFrame(dataPack1.data)
df

,0,1,2
0,123,0x79Be957bf7e3003aFd0e78f04Bacbc93D3ef2fB7,2023-07-15 02:25:32.392441
1,124,0x79Be957bf7e3003aFd0e78f04Bacbc93D3ef2fB7,2023-07-15 02:25:32.392441
2,125,0x79Be957bf7e3003aFd0e78f04Bacbc93D3ef2fB7,2023-07-15 02:25:32.392441


- Save the data into a table in MindLake

In [6]:
mindlake = mindlakesdk.connect(env.walletPrivateKey, env.mindLakeAppKey, env.MINDLAKE_GATEWAY)
assert mindlake, mindlake.message

# drop the table if exists
result = mindlake.datalake.dropTable('a_new_table')
print("dropTable result:", result.message)

# upload from local csv into mindlake
result = dataPack1.saveToMindLake('a_new_table', mindlake)
assert result, result.message
print("a_new_table has been saved to MindLake.")

dropTable result: Success
a_new_table has been saved to MindLake.


you can also check this new downloaded table in https://scan.mindnetwork.xyz/account/myData

## Step 3. MindLake -> Arweave
- Load data from a table in MindLake and save to Arweave

In [7]:
dataPack2 = DataPack(env.walletPrivateKey)
# make sure if you complete Step 1, so "a_new_table" exists in mindlake 
result = dataPack2.loadFromMindByQuery('select * from "a_new_table"', mindlake)
assert result, result.message
df = pandas.DataFrame(dataPack2.data)
df

,0,1,2
0,123,0x79Be957bf7e3003aFd0e78f04Bacbc93D3ef2fB7,2023-07-15 02:25:32.392441
1,124,0x79Be957bf7e3003aFd0e78f04Bacbc93D3ef2fB7,2023-07-15 02:25:32.392441
2,125,0x79Be957bf7e3003aFd0e78f04Bacbc93D3ef2fB7,2023-07-15 02:25:32.392441


- Save the data into Arweave

In [8]:
# make sure you have arweave_wallet.json 
result = dataPack2.saveToArweave('test_table_encrypted.csv', 'ACNH')
# result = dataPack2.saveToArweave('test_table_encrypted.csv', 'AR', './arweave_wallet.json')
assert result, result.message
print("test_table_encrypted.csv has been saved to Arweave.\n")
itemID = result.data
print("The Arweave ID is:", itemID, '\n')
arweave_explore_url = "https://viewblock.io/arweave/tx/%s"%(itemID)
arweave_file_url = "https://arseed.web3infra.dev/%s"%(itemID)
print(f"You can check on Arweave: {arweave_explore_url}\n")
print(f"And the content on Arweave: {arweave_file_url}\n")
import requests
content = requests.get(arweave_file_url).text
print("The content is:")
print(content)

test_table_encrypted.csv has been saved to Arweave.

The Arweave ID is: sMboxrf5U9Z-FM_-woSfiVerz0Lzpa1_OCmRX09Jgn0 

You can check on Arweave: https://viewblock.io/arweave/tx/sMboxrf5U9Z-FM_-woSfiVerz0Lzpa1_OCmRX09Jgn0

And the content on Arweave: https://arseed.web3infra.dev/sMboxrf5U9Z-FM_-woSfiVerz0Lzpa1_OCmRX09Jgn0

The content is:
uid,wallet_address,register_date
123,\x9326bce223092712067ca9dd8d6600fd6d0a8e572261927932f0b222092249804635af229e5bfd2e8ed550fb44539c37fa5f4bc77e48499ce01ed9bec037ea7ddd,\x992afbd2992fcc46071972b3069b44476bc2316cacbe12a5305839d60ab43dc674
124,\x7726950c54dfd2470b9ef8929ee3c049af498bf99a56756f1824f0146fcda22df0bb392959e42a2c8639604ec301425e31844b8a7b847ae486abd29d6594c0ff7c,\xcdf8811927b22d47aa10e2e8f367e986461d43242b9f81ff62564424d64f958235
125,\x72ccfbbd92144aed8bd0763e30f63e68ac99f095a5fdcc3772ea4b35ead34ede8fc891e7b8e73ae5d07e25fc3b801e66bbea0a110e9047b3bccfdddeac8f7cbd51,\xb8138ce9e9feaff5c610f424cc97dfb44d1809ad31ca830fd15039d9d3124679ac



In [12]:
import base64


bytes.fromhex(env.walletPrivateKey)
base64.b64encode(bytes.fromhex(env.walletPrivateKey))

b'Kndi9N13b3f8ZeL3oJU7HreK+Oxibo0OfvYLAQuLsU4='

## Step 4. Arweave -> Local File
- Load data from Arweave and save to local file

In [9]:
dataPack3 = DataPack(env.walletPrivateKey)
result = dataPack3.loadFromArweave(itemID)
assert result, result.message
print("You can see file on Arweave are encrypted, but datapack can decrypted it locally")
df = pandas.DataFrame(dataPack3.data)
df

You can see file on Arweave are encrypted, but datapack can decrypted it locally


,0,1,2
0,123,0x79Be957bf7e3003aFd0e78f04Bacbc93D3ef2fB7,2023-07-15 02:25:32.392441
1,124,0x79Be957bf7e3003aFd0e78f04Bacbc93D3ef2fB7,2023-07-15 02:25:32.392441
2,125,0x79Be957bf7e3003aFd0e78f04Bacbc93D3ef2fB7,2023-07-15 02:25:32.392441


- Save the data into a local file with encryption

In [10]:
result = dataPack3.saveToLocalFile('test_localtable_encrypted.csv', False)
assert result, result.message
print("test_localtable_encrypted.csv has been saved to local file.")
df = pandas.read_csv('test_localtable_encrypted.csv')
df

test_localtable_encrypted.csv has been saved to local file.


,uid,wallet_address,register_date
0,123,\x9c851ae97fd6eeb3ae2437c274274b3d773fcdee11f9...,\x44e6bdad517e634a985702320c26770a9275c79aa9b3...
1,124,\xaddba1e7648b04018dabaceb918cebb436dff5e57e61...,\xb5dac593d9436309272b5b56c16d4db24bab7d9e130a...
2,125,\x9afc9f24ec67f79105a340e939be6f1808f5a34495f2...,\x755479852b673a2183e1168734e78378fcf81413c3f2...
